# Using Python to Create Useful Raster Data from Climate.ca's Climate Variables  

[Climatedata.ca](https://www.climatedata.ca) is the product of a collaboration between Environment and Climate Change Canada (ECCC), the Computer Research Insitute of Montréal, Ouranos, the Pacific Climate Impacts Consortium (PCIC), the Prairie Climate Centre (PCC), and HabitatSeven.  

Climatedata.ca provides free and open access to climate data. This includes both future climate projections and historical data. [Instructions](https://climatedata.ca/learn/#module-5) to download climata data are provided on the website.  

The climate data is provided in 10x6 point layers, with points for each year superimposed upon each other.

Using the Time Series activation properties in ArcGIS Pro, a user can scroll through the data and see how the points change over time.  
 
However, these small points are quite difficult to see. It is also difficult to infer how a the value of a point in between the visible points might differ from the recorded values.  

This means, if someone were to ask us, "What was the level of precipitation at X point in Y year", we would have to hope that the X point fell perfectly into the same location as a point in the data. This is very unlikely!  

To fix this problem, we set out to create a Python script that would produce raster datasets of each climate variable for each year. We will use two types of interpolation: Kriging, and Empirical Bayesian Kriging, to examine the results of each method.

## The data
The data used for this project was focused on the Annapolis County in Nova Scotia, Canada. Climatedata.ca was used to batch-download 35 different climate variables covering the Annapolis County. The area of the county required 78 points to cover. The data included historical data as far back as 1950, and climate projection up until 2100 (151 years total). All climate variables were depicted across three different climate scenarios (RCP 2.6, RCP 4.5, and RCP 8.5). [LINK HERE]()

The resulting 35 point layers each contained 11,778 points (151 x 78 = 11,778) across 78 locations. In other words, the 78 data points for each year from 1950 to 2100 were stacked on top of each other in each climate data layer. 

# Step 0: Import libraries and set environment parameters

In [1]:
import os 
import arcpy, datetime
from arcpy.sa import *

In [2]:
arcpy.env.overwriteOutput = True

# Step 1: Create a point layer for each year of data

Instead of having one layer with 11,778 points representing 151 years of data for each climate variable, it would be more useful to have a layer representing each climate variable for a particular year.

We will create a new layer for each year, from 1950 to 2100, for each climate variable. 

We will also create a new geodatabase for all of these points to go into.  

<a id='Climate_Points_Output_Geodatabase'></a>

In [3]:
# Set the location where all the original climate variable layers are stored
file_gdb = r'D:\COGS_Term2\ACFA_TermProject\ClimateScripting\final_test.gdb'

In [4]:
# Set "Parent" folder where all folders and Geodatabases will output
gdb_folder = r'D:\COGS_Term2\ACFA_TermProject\ClimateScripting'

In [22]:
# Set the name for the output geodatabase where all 151 output layers
# for each climate variable should go
out_gdb_name = 'Climate_final_test_output.gdb'

In [23]:
# Set the location for a new geodatabase where all 151 output layers 
# for each climate variable should go
output_gdb = os.path.join(gdb_folder, out_gdb_name)

In [27]:
# Create output geodatabase where all 151 output layers for each climate
# variable should go
if arcpy.Exists(output_gdb) == False:
    arcpy.management.CreateFileGDB(gdb_folder, out_gdb_name)

In [9]:
# Set the workspace to the location where all the original climate variable 
# layers are stored
arcpy.env.workspace = file_gdb

In [10]:
# Store a list of all the feature classes (all the point layers) that we 
# will be putting through this script.
fclasses = arcpy.ListFeatureClasses()

In [11]:
# View the list of all feature classes to be manipulated.
fclasses

['all_BCCAQv2_var_annual_tx_mean_FC_elev']

### Step 1.1: Re-project point layers to UTM
We want to convert the data from WGS 84 to UTM Zone 20N for use in future projects. 

In [12]:
# Name of Geodatabase for projected data
gdb_name = 'Climate_final_test_Project.gdb'

In [13]:
# Full file path of Geodatabase for projected data
project_gdb = os.path.join(gdb_folder, gdb_name)

In [14]:
# Create new GDB for projected data
if arcpy.Exists(project_gdb) == False:
    arcpy.management.CreateFileGDB(gdb_folder, gdb_name)

In [15]:
# Re-project data
for fclass in fclasses:
    out_coord = arcpy.SpatialReference('NAD 1983 CSRS UTM Zone 20N')
    out_fclass = os.path.join(project_gdb, fclass)
    arcpy.management.Project(fclass, out_fclass, out_coord)

### Step 1.2 Create a point layer for every year of data

In [16]:
# Re-set workspace to GDB with projected data
arcpy.env.workspace = project_gdb
# Update the fclasses variable to store the list of new projected point layers
fclasses = arcpy.ListFeatureClasses()

In [21]:
for fclass in fclasses:
    print(fclass.split('_elev')[0])

all_BCCAQv2_var_annual_tx_mean_FC


In [28]:
# Create a point layer for every year of data

try:
    for fclass in fclasses:
        year = 1950
        fc_time = "timestamp '{}-01-01 00:00:00'".format(year)
        print('The feature class name is: {}'.format(fclass))
        while year <= 2100:
            print('Selecting points with {} in feature class'.format(fc_time))
            
            # create an sql where statement to find the date
            where = "time = " + "%s" %fc_time
            
            # split the fclass name for a nicer layer name
            lyr = fclass.split('_elev')[0]

            # Make a layer from the feature class
            arcpy.management.MakeFeatureLayer(fclass, lyr)
            
            # select statement using our where statement
            arcpy.SelectLayerByAttribute_management(lyr, "NEW_SELECTION", where)
            
            # create a file name by adding the time to the fclass name
            f_name = lyr + str(year)
            print('The name of the new layer made from selected points is: {}'.format(f_name))
            # create a file path for export to the output gdb using the new file name
            f_path = os.path.join(output_gdb, f_name)
            print('File path for new layer: {}'.format(f_path))
            
            # Write the selected features to new featureclass using file path
            arcpy.CopyFeatures_management(lyr, f_path)
            #fc_time = fc_time + timedelta(days=365)
            year = year + 1
            fc_time = "timestamp '{}-01-01 00:00:00'".format(year)

except:
    print(arcpy.GetMessages())
        
# read more about datetime module: https://docs.python.org/2/library/datetime.html#strftime-and-strptime-behavior



The feature class name is: all_BCCAQv2_var_annual_tx_mean_FC_elev
Selecting points with timestamp '1950-01-01 00:00:00' in feature class
The name of the new layer made from selected points is: all_BCCAQv2_var_annual_tx_mean_FC1950
File path for new layer: D:\COGS_Term2\ACFA_TermProject\ClimateScripting\Climate_final_test_output.gdb\all_BCCAQv2_var_annual_tx_mean_FC1950
Selecting points with timestamp '1951-01-01 00:00:00' in feature class
The name of the new layer made from selected points is: all_BCCAQv2_var_annual_tx_mean_FC1951
File path for new layer: D:\COGS_Term2\ACFA_TermProject\ClimateScripting\Climate_final_test_output.gdb\all_BCCAQv2_var_annual_tx_mean_FC1951
Selecting points with timestamp '1952-01-01 00:00:00' in feature class
The name of the new layer made from selected points is: all_BCCAQv2_var_annual_tx_mean_FC1952
File path for new layer: D:\COGS_Term2\ACFA_TermProject\ClimateScripting\Climate_final_test_output.gdb\all_BCCAQv2_var_annual_tx_mean_FC1952
Selecting points

# Step 2: Create a raster for each point file layer

We will create a geodatabase for each climate variable. This will ensure that the rasters for each climate variable are stored in their own dedicated place for future use.  
Need a reminder of the file path for output_gdb? Return to [this section](#Climate_Points_Output_Geodatabase).

In [29]:
# Change the workspace to the geodatabase containing the feature classes 
# representing each year for each climate variable that we just created
arcpy.env.workspace = output_gdb

In [30]:
# Store a list of all the 151 point layers for each climate variable created
# This will be a long list!
new_fclasses = arcpy.ListFeatureClasses()

In [31]:
# Check the length of the list we just created
len(new_fclasses)

151

### Step 2.1: Selecting the climate scenario
As previously discussed, all values for each climate variable are available across three scenarios, or RCP's.  
RCP stands for 'Representative Concentration Pathway'. The RCP's try to predict how human activities will impact concentrations of greenhouse gases in the atmosphere. For example, will we continue to burn fossil fuels, or will we shift towards renewable energy? 
- RCP 2.6: With **high levels** of effort to curb emissions, the RCP2.6 pathway is achieved. A 1.0&deg;C average increase in temperature is expected, below the threshold at which climate change becomes dangerous (2&deg;C).
- RCP 4.5: With **medium** levels of effort to curb emissions, the RCP4.5 pathway is acheived. A 2.2&deg;C average increase in temperature is expected, above the threshold at while climate change becomes dangerous (2&deg;C).
- RCP 8.5: With **low** levels of effort to curb emissions, the RCP8.5 pathway is acheived. Current emissions are tracking close to the RCP8.5 pathway. A 3.7&deg;C average increase in temperature is expected, above the threshold at which climate change becomes dangerous (2&deg;C)

We will now choose which scenario we will model in our rasters.

In [32]:
# Choose climate scenario: one of rcp26, rcp45, or rcp85
rcp = "rcp85"
# Choose model percentile
p = "p90"

### Step 2.2: Ordinary kriging

In [33]:
# Create a file name for the new folder to hold all unique geodatabases
# for each climate feature undergoing ordinary kriging
ord_kriging_folder = 'Ordinary_Kriging_final_test_Output_Geodatabases'

In [34]:
# Create a new folder to hold all unique geodatabases for each 
# climate feature
if arcpy.Exists(ord_kriging_folder) ==False:
    arcpy.management.CreateFolder(gdb_folder, ord_kriging_folder)

In [35]:
# Set the path for the new folder that will hold all unique
# geodatabases for each climate feature
out_folder = os.path.join(gdb_folder, ord_kriging_folder)

In [37]:
for fc in new_fclasses:
    fields = [f.name for f in arcpy.ListFields(fc)]
    filter_field = filter(lambda a: a.startswith(rcp) and a.endswith(p), fields)
    field = list(filter_field)
    field_str = ''.join(map(str, field))
    print(field_str)

rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_mean_p90
rcp85_tx_m

In [40]:
# Create a new file geodatabase for each climate feature
# Perform ordinary kriging on each point layer created for 
# each year of each climate feature
# Save all new rasters to the appropriate geodatabases

try:
    for fc in new_fclasses:
        fields = [f.name for f in arcpy.ListFields(fc)]
        filter_field = filter(lambda a: a.startswith(rcp) and a.endswith(p), fields)
        field = list(filter_field)
        field_str = ''.join(map(str, field))
        f_name = fc + "_{}{}".format(rcp,p)
        gdb_name = fc[4:-7] + '.gdb'
        gdb_path = os.path.join(out_folder, gdb_name) 
        if arcpy.Exists(gdb_path) == False:
            print('Creating file geodatabase...')
            arcpy.management.CreateFileGDB(out_folder, gdb_name) 
            print('GDB file path: {}'.format(gdb_path))
        out_path = os.path.join(gdb_path, f_name)
        #out_surface_raster = Kriging(fc, field_str, KrigingModelOrdinary("Spherical # # # #", 0.003, "VARIABLE 12", None))
        #yonkers
        #out_surface_raster = arcpy.sa.Kriging(fc, field_str, "Spherical # # # #", 44966.1061417827, "VARIABLE 12", None)
        out_surface_raster = arcpy.sa.Kriging(fc, field_str, "Spherical # # # #", 0.003, "VARIABLE 12", None)
        out_surface_raster.save(out_path)
        print('Raster added to GDB: {}'.format(f_name))
except:
    print(arcpy.GetMessages())

Start Time: May 19, 2022 6:57:24 AM
ERROR 010092: Invalid output extent.
Failed to execute (Kriging).
Failed at May 19, 2022 6:57:24 AM (Elapsed Time: 0.46 seconds)


### Step 2.3: Empirical Bayesian Kriging (EBK)

In [23]:
# Create a file name for the new folder to hold all unique geodatabases
# for each climate feature undergong Empirical Bayesian Kriging (EBK)
ebk_folder = 'EBK_Output_Geodatabases'

In [25]:
# Create a new folder to hold all unique geodatabases for each 
# climate feature
arcpy.management.CreateFolder(gdb_folder, ebk_folder)

<Result 'D:\\COGS_Term2\\ACFA_TermProject\\ClimateScripting\\EBK_Output_Geodatabases'>

In [26]:
# Set the path for the new folder that will hold all unique
# geodatabases for each climate feature
out_ebk_folder = os.path.join(gdb_folder, ebk_folder)

In [33]:
for fc in new_fclasses:
    name = fc.split('all_BCCAQv2_var_annual_')[1].replace('_','').replace('FC','')
    print(name)

tnmean1950
tnmean1951
tnmean1952
tnmean1953
tnmean1954
tnmean1955
tnmean1956
tnmean1957
tnmean1958
tnmean1959
tnmean1960
tnmean1961
tnmean1962
tnmean1963
tnmean1964
tnmean1965
tnmean1966
tnmean1967
tnmean1968
tnmean1969
tnmean1970
tnmean1971
tnmean1972
tnmean1973
tnmean1974
tnmean1975
tnmean1976
tnmean1977
tnmean1978
tnmean1979
tnmean1980
tnmean1981
tnmean1982
tnmean1983
tnmean1984
tnmean1985
tnmean1986
tnmean1987
tnmean1988
tnmean1989
tnmean1990
tnmean1991
tnmean1992
tnmean1993
tnmean1994
tnmean1995
tnmean1996
tnmean1997
tnmean1998
tnmean1999
tnmean2000
tnmean2001
tnmean2002
tnmean2003
tnmean2004
tnmean2005
tnmean2006
tnmean2007
tnmean2008
tnmean2009
tnmean2010
tnmean2011
tnmean2012
tnmean2013
tnmean2014
tnmean2015
tnmean2016
tnmean2017
tnmean2018
tnmean2019
tnmean2020
tnmean2021
tnmean2022
tnmean2023
tnmean2024
tnmean2025
tnmean2026
tnmean2027
tnmean2028
tnmean2029
tnmean2030
tnmean2031
tnmean2032
tnmean2033
tnmean2034
tnmean2035
tnmean2036
tnmean2037
tnmean2038
tnmean2039
tnmean2040

In [35]:
# Perform Empirical Bayesian Kriging

try:
    for fc in new_fclasses:
        fields = [f.name for f in arcpy.ListFields(fc)]
        filter_field = filter(lambda a: a.startswith(rcp) and a.endswith(p), fields)
        field = list(filter_field)
        field_str = ''.join(map(str, field))
        f_name = fc.split('all_BCCAQv2_var_annual_')[1].replace('_','').replace('FC','')
        gdb_name = fc[4:-7] + '_{}{}_EBK.gdb'.format(rcp,p) 
        gdb_path = os.path.join(out_ebk_folder, gdb_name) 
        if arcpy.Exists(gdb_path) == False:
            print('Creating file geodatabase...')
            arcpy.management.CreateFileGDB(out_ebk_folder, gdb_name) 
            print(gdb_path)
        out_path = os.path.join(gdb_path, f_name)
        arcpy.ga.EmpiricalBayesianKriging(fc, field_str, None, out_path, 319.0023464, "NONE", 100, 1, 100, "NBRTYPE=StandardCircular RADIUS=28770.3543520455 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "PREDICTION", 0.5, "EXCEED", None, "POWER")
        print(out_path)
except:
    print(arcpy.GetMessages())

Creating file geodatabase...
D:\COGS_Term2\ACFA_TermProject\ClimateScripting\EBK_Output_Geodatabases\BCCAQv2_var_annual_tn_mean_rcp85p90_EBK.gdb
D:\COGS_Term2\ACFA_TermProject\ClimateScripting\EBK_Output_Geodatabases\BCCAQv2_var_annual_tn_mean_rcp85p90_EBK.gdb\tnmean1950
D:\COGS_Term2\ACFA_TermProject\ClimateScripting\EBK_Output_Geodatabases\BCCAQv2_var_annual_tn_mean_rcp85p90_EBK.gdb\tnmean1951
D:\COGS_Term2\ACFA_TermProject\ClimateScripting\EBK_Output_Geodatabases\BCCAQv2_var_annual_tn_mean_rcp85p90_EBK.gdb\tnmean1952
D:\COGS_Term2\ACFA_TermProject\ClimateScripting\EBK_Output_Geodatabases\BCCAQv2_var_annual_tn_mean_rcp85p90_EBK.gdb\tnmean1953
D:\COGS_Term2\ACFA_TermProject\ClimateScripting\EBK_Output_Geodatabases\BCCAQv2_var_annual_tn_mean_rcp85p90_EBK.gdb\tnmean1954
D:\COGS_Term2\ACFA_TermProject\ClimateScripting\EBK_Output_Geodatabases\BCCAQv2_var_annual_tn_mean_rcp85p90_EBK.gdb\tnmean1955
D:\COGS_Term2\ACFA_TermProject\ClimateScripting\EBK_Output_Geodatabases\BCCAQv2_var_annual_tn

# Step 3: Adding the Rasters to Mosaic Datasets

At this point we will begin adding our rasters to Raster Mosaic Datasets.  
We will use a format that will allow Time Series activation later on.

## Step 3.1 Adding the Rasters to Mosaic Datasets: Ordinary Kriging

In [8]:
# Change the workspace to the folder that now contains the geodatabases for 
# each feature
arcpy.env.workspace = out_folder

In [9]:
# Store a list of all workspaces within our current workspace
ws = arcpy.ListWorkspaces()
# Display a list of all workspaces within our current workspace
ws

In [2]:
# Create a name for the new Geodatabase that we will create to hold 
# all Raster Mosaic Datasets
gdb_name = 'ClimateMosaics_OrdinaryKriging.gdb'

In [12]:
# Create a new file gdb to contain the final mosaic datasets
arcpy.management.CreateFileGDB(out_folder, gdb_name)

<Result 'D:\\COGS_Term2\\ACFA_TermProject\\ClimateScripting\\Test_Outputs2\\ClimateMosaics.gdb'>

In [6]:
# Set the file path for the new Geodatabase that will hold all 
# Raster Mosaic Datasets
out_gdb = os.path.join(out_folder, gdb_name)

### Step 3.1.1 Creating a raster mosaic dataset for each climate variable and adding rasters

In [14]:
# Create mosaic dataset for each feature and import all relevant rasters
# If mosaic dataset already exists due to re-running data, delete and recreate
# Add rasters to new mosaics

for w in ws:
    print('Current geodatabase: {}'.format(w))
    m_name = w.split('\\')[5][:-4]
    m_path = os.path.join(out_gdb, m_name)
    coord = arcpy.SpatialReference("NAD 1983 CSRS UTM Zone 20N")
    if arcpy.Exists(m_path):
        arcpy.management.DeleteMosaicDataset(m_name)
    arcpy.management.CreateMosaicDataset(out_gdb, m_name, coord)
    arcpy.management.AddRastersToMosaicDataset(m_path, "Raster Dataset", w, "UPDATE_CELL_SIZES", "UPDATE_BOUNDARY", "UPDATE_OVERVIEWS", None, 0, 1500, None, '', "SUBFOLDERS", "ALLOW_DUPLICATES", "NO_PYRAMIDS", "CALCULATE_STATISTICS", "NO_THUMBNAILS", '', "NO_FORCE_SPATIAL_REFERENCE", "ESTIMATE_STATISTICS", None, "NO_PIXEL_CACHE")
    print('Added rasters from current geodatabase into new Raster Mosaic Dataset.')
    print('Location of new Raster Mosaic Dataset: {}'.format(out_gdb))
    print('Name of new Raster Mosaic Dataset: {}'.format(m_name))

D:\COGS_Term2\ACFA_TermProject\ClimateScripting\Test_Outputs2\BCCAQv2_var_annual_tn_mean.gdb


In [7]:
# Change workspace to Climate Mosaics gdb
arcpy.env.workspace = out_gdb

In [8]:
# Store a list of all datasets with the Climate Mosaics geodatabase
mosaics = arcpy.ListDatasets()
# Display the list of all datasets within the Climate Mosaics geodatabase
# We should see a list of all of our Raster Mosaic Datasets
mosaics

['BCCAQv2_var_annual_tn_mean']

### Step 3.2.2 Add a Time field and fill it with the appropriate year
This will allow us to use time series activation with each raster. This is the final step of the process for Ordinary Kriging!

In [ ]:
# https://gis.stackexchange.com/questions/60810/add-a-time-dimension-in-esri-mosaic-raster-dataset

for m in mosaics:
    lst = []
    table_name = 'AMD_' + m + '_CAT'
    t = os.path.join(out_gdb, table_name)
    fields = arcpy.ListFields(t)
    for field in fields:
        lst.append(field.name)
    if 'Time' not in lst:
        arcpy.management.AddField(t, 'Time', 'SHORT')
        print("Added 'Time' field.")
    rows = arcpy.UpdateCursor(t, ['*'])
    for row in rows:
        year = int(row.name.split('_FC')[1][0:4])
        #timestring= datetime.datetime(year, 1, 1)
        row.time = year
        rows.updateRow(row)
        print(row.time)
    


## Conclusions

Now that we have a raster dataset for each year for each climate variable, we can readily perform calculations and visualize changes. If we want to examine the average temperature change between two decades, for example, we can do so while viewing interpolated values across the whole study area. This script can support future academic or other research, as it provides new dimensions of use for the data at ClimateData.ca.